   # Training machines to smell - Part 1: Data acquisition and feature generation.

## Project overview

This project is dedicated to create machine learning models for predicting odor properties of molecules from the DREAM Olfactory Challenge. In this first notebook, the data acquisition and feature generation is discussed. For the sake of organization, the training procedure and exploratory data analysis will be implemented in other notebook files.

## Read the data

In [1]:
# Importing libraries 
import pandas as pd
import os

The cell below downloads the dataset from [this repository](https://github.com/dream-olfaction/olfaction-prediction). The repo contains data used for DREAM Olfaction Prediction Challenge. The data collected from the experiments were splitted into 3 smaller datasets. In the competition, the first was intended to be used for training the machine learning algorithm, the second to evaluate the performance of the competitors half-way the end of the competition and the third to create the final rank and rank competitors. 

All three data set are in the ``data`` directory. The first dataset is in the ``TrainSet.txt`` file, the second in the ``leaderboard_set.txt`` and the third in the ``TextSet.txt``.

I have not cloned the directory to the jupyter notebook instance since there are a lot of additional files that I will not use. Instead, I will load them directly from the github repo using pandas.

In [2]:
!pip install xlrd==1.2.0

     |████████████████████████████████| 103 kB 20.8 MB/s eta 0:00:01
  Attempting uninstall: xlrd
    Found existing installation: xlrd 2.0.1
    Uninstalling xlrd-2.0.1:
      Successfully uninstalled xlrd-2.0.1


In [3]:
def txt2df(file_path):
    ''' Reads a raw file from a github online repository and store it in a pandas dataframe.
    
        param file_path: a string that points to a .txt file in a github repo (It has to be a raw file)
        return: Pandas dataframe with the content inside the file
    '''
    df = pd.read_table(file_path) #reades .txt file format
    return df

In [4]:
#Defining online github directory path and file names
data_ondir = 'https://raw.githubusercontent.com/dream-olfaction/olfaction-prediction/master/data/'
train_file = 'TrainSet.txt' #train set file name
ldboard_file = 'LBs1.txt'   #Leaderbord file name subchallenge 1
#ldboard_file = 'leaderboard_set.txt'
test_file = 'TestSet.txt'

#Joining strings to compose the path that points to the files
train_path = os.path.join(data_ondir, train_file)
ldboard_path = os.path.join(data_ondir, ldboard_file)
test_path = os.path.join(data_ondir, test_file)

In [5]:
#Reading the files from the repo and storing in a dataframe
train_df = txt2df(train_path)
ldb_df = txt2df(ldboard_path)
test_df = txt2df(test_path)

### Metadata
* The first column (“Compound Identifier”) contains the PubChem Compound Identification, an identifier for a unique chemical structure.
*  The second column (“Odor”) contains a synonym for the chemical. Be aware that there are many synonyms. For identifying the odor, the CID and not the odor name should be used.
*  The third column ("replicate") indicates whether the stimulus is part of the 20 stimuli that were tested twice.
* The fourth column (“intensity”) indicates whether the data is for the odor at high intensity (lower dilution of that odor) or at low intensity (higher dilution of that odor).
* The fifth column (“dilution”) contains the dilution of the odor. Each odor is presented at two different dilutions. Weak odors are diluted less than strong odors, so that the low dilution of all the odors has approximately equal intensity. The high dilution of all the odors also has approximately equal intensity. (Equi-intensity is only very roughly approximated. Some of the chemicals have no smell at all, so they obviously cannot be diluted to be equi-intense with others.)
* The sixth column (“subject #”) contains a subject identifier, a number from 1 to 49.

Columns 7-27 contain perceptual data that the 49 subjects provided during smell testing. Before subjects rated a given odor, they were asked if they could smell anything when they sniffed the odor. If they indicated that they smelled nothing, the intensity rating was automatically set to “0” and the other ratings were left blank. Occasionally, subjects answered this first question yes, indicating that they could smell something, but then proceeded to rate the intensity as “0”. These subjects also completed all the other ratings. This is why you will find that in most cases when the intensity rating is “0,” there are no other data, but in a few cases you will find a complete data set even if the intensity was rated “0.”

* The seventh column (“Intensity/Strength”) contains perceptual data about how intense or strong the smell was perceived by the subjects. Subjects used a scale from 0-100 where 100 is “extremely strong” and 0 is “extremely weak”.
* The eighth column (“Valence/Pleasantness”) contains perceptual data about how pleasant or unpleasant the smell was perceived by the subjects. Subjects used a scale from 0-100 where 100 is “extremely pleasant” and 0 is “extremely unpleasant”.
* Columns 9-27 contain data in which subjects matched their perception of how the odor smelled to a standard list of 19 perceptual descriptors: bakery, sweet, fruit, fish, garlic, spices, cold, sour, burnt, acid, warm, musky, sweaty, ammonia/ruinous, decayed, wood, grass, flower, chemical

*Subjects used a scale from 0-100 where 100 is “very much” and 0 is “not at all”.*

##  Feature generation.

The datasets do not provide informative data about the molecules except its commonly used name and identifier. One way to gather more information about them is by using their PubChem Compound Identification and lookup their features in the PubChem database.

Let's just make some definitions before diving into molecular features

* ``molecules`` are  a collection of atoms connected by chemical bonds in a 3-D arrangement. (That's my definition)

So, we need a set of informations to fully define a molecule: The position of the atoms with respect to a set of coordinates (x,y,z), which atom are in each position and how they connect with their neighboors (bonds). Basically, we are encoding molecules using symbolic entities -- real numbers (position of atoms) and letters (C for carbon, O for oxygen). 


However, this is not the only way to model molecules with symbolic entities. There is a famous symbolic representation of molecules called SMILES (simplified molecular-input line-entry system). Molecules are represented with lined ASCII strings which aim to define a molecule from its features (atoms' type, geometric distribution and connectivity). The rules to model molecules with SMILES can found [here](https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system). Just to give a concrete example, water can be represented as "O" or by "[H]O[H]". One of the characteristics of this representation is that a single molecule can have multiple SMILES representation, but a given SMILES identify one and only molecule.


We can go from symbolic entities to useful numeric representations! The repository from which I collected the data sets also contain [Dragon descriptors](https://match.pmf.kg.ac.rs/electronic_versions/Match56/n2/match56n2_237-248.pdf) for all the molecules! They compute numerical representations of molecules from their atomic constituents. 
These descriptors is shown to be of similar quality (statistically indistinguishable) compared with Mordred and Bitbased RDKit fingerprints for [QSOR problems](https://arxiv.org/abs/1910.10685).

### Training set

* In this section, the code is used to  generate features for the training set

**Dropping rows with NaN (null odor intensity)**

``Before`` splitting the data I will deal is deal with missing entries in the data set. As the data description shows, there are missing data. The missing data are cases where the subjects could not smell anything at all in the experiment. These rows have also a 0 intensity entry. In face of that fact, inputation of missing values does not make sense. The only way to deal with it is by removing experiments where the subject was not sensitive to a particular smell at a particular dilution.  

In [5]:
print('Raw data set contains', train_df.shape[0], 'rows')

Raw data set contains 35084 rows


* Dropping rows with ``pd.DataFrame.dropna()``

In [6]:
train_df.drop(columns = ['Odor', 'Replicate'], inplace=True)

In [7]:
train_df.dropna(axis = 0, how = 'any', inplace = True)

In [8]:
print('After dropping NaN rows it has', train_df.shape[0], 'rows')

After dropping NaN rows it has 25980 rows


In [9]:
print('A total of', 35084 - train_df.shape[0], 'experiments were removed.',  (1 - train_df.shape[0]/35084)*100, '%') 

A total of 9104 experiments were removed. 25.949150609964654 %


* Selecting the column in the training set with molecular identifiers:

In [10]:
train_ids = train_df[['Compound Identifier']]

**Importing Features**

Now I almost have the data in the format I want. The next step is transforming the molecular models into useful numeric representations! The repository from which I collected the data sets also contain [Dragon descriptors](https://match.pmf.kg.ac.rs/electronic_versions/Match56/n2/match56n2_237-248.pdf) for all the molecules! These descriptors is shown to be of similar quality (statistically indistinguishable) compared with Mordred and Bitbased RDKit fingerprints for [QSOR problems](https://arxiv.org/abs/1910.10685).

Importing Dragon features:

In [6]:
dragon1 = pd.read_table('https://raw.githubusercontent.com/dream-olfaction/olfaction-prediction/master/data/molecular_descriptors_data.txt')
dragon2 = pd.read_excel('data/dragon descriptors for 0869 odors.xlsx')

In [7]:
dragon = pd.concat([dragon1,dragon2], axis = 0)

In [8]:
dragon = dragon.iloc[:, 0:-2]

Creating a directory to store the features as csv file

Now we can create our feature matrix for training set by looking up the dragon the descriptors of the compound identifier column. Additionally, I will save both features and targets as csv files in hard disk to free memory.

In [14]:
if not os.path.isdir('task1'):
    os.mkdir('task1')
if not os.path.isdir('task2'):
    os.mkdir('task2')

* Look up  ``mol_ids`` features with ``pd.DataFrame.merge`` method. Left data frame is ``train_ids`` and right dataframe is ``dragon``

In [15]:
#Look up
X = train_ids[['Compound Identifier']].merge(dragon, left_on = 'Compound Identifier', 
                                                  right_on = 'CID').iloc[:,1:]
#print('number of features is', X.shape[1])

In [16]:
print('number of features is', X.shape[1])

number of features is 4870


Labels for task 1

In [17]:
y_1 = train_df.loc[:,['subject #', 'INTENSITY/STRENGTH']]
y_1.reset_index(inplace = True, drop = True)

In [18]:
print(X.shape, y_1.shape)

(25980, 4870) (25980, 2)


Labels for task 2

In [19]:
cols2drop = ['Compound Identifier', 'Intensity','Dilution','INTENSITY/STRENGTH']
y_2 = train_df.drop(columns = cols2drop)
y_2.reset_index(inplace = True, drop = True)

In [20]:
y_2.columns

Index(['subject #', 'VALENCE/PLEASANTNESS', 'BAKERY', 'SWEET', 'FRUIT', 'FISH',
       'GARLIC', 'SPICES', 'COLD', 'SOUR', 'BURNT', 'ACID', 'WARM', 'MUSKY',
       'SWEATY', 'AMMONIA/URINOUS', 'DECAYED', 'WOOD', 'GRASS', 'FLOWER',
       'CHEMICAL'],
      dtype='object')

After the look up, some features are missing in the training set. The following cells drop rows with NaNs

In [21]:
#Creating a boolean mask for NaN values
rows_mask = X.isnull().any(axis = 1)

In [22]:
#Applying the mask both in X and y dataframes
X, y_1, y_2  = X.loc[~rows_mask], y_1.loc[~rows_mask], y_2.loc[~rows_mask]

In [23]:
print(X.shape, y_1.shape, y_2.shape)

(25762, 4870) (25762, 2) (25762, 21)


Saving ``X``, ``y_1`` and ``y_2`` to csv.

In [24]:
X.to_csv('task1/X_train.csv', index = False)
y_1.to_csv('task1/y_train.csv', index = False)
y_2.to_csv('task2/y_train.csv', index = False)

In [26]:
#cleaning memory
train_df = None
X = y_1 = y_2 =  None

###  Leaderboard and Test set

In my analysis, I will use the leaderboard set as my validation set eventhough this is was not what happened during the challenge.

**Leaderboard**

**Dropping rows with NaN**

In [37]:
ldb_df.columns

Index(['#oID', 'individual', 'descriptor', 'value'], dtype='object')

NaN mask

In [38]:
na_mask = ldb_df.isna().value

In [39]:
ldb_df = ldb_df[~na_mask]

Unpivoting the dataframe

In [40]:
df2 = ldb_df.pivot_table(index=['#oID','individual'], columns='descriptor', aggfunc= lambda x: x)
df2.columns = df2.columns.droplevel().rename(None)
df2 = df2.reset_index().dropna(axis = 0, how = 'any')

In [42]:
y1_ldb = df2[['individual', 'INTENSITY/STRENGTH']]
cols2 = ['individual','VALENCE/PLEASANTNESS', 'BAKERY', 'SWEET', 'FRUIT', 'FISH',
       'GARLIC', 'SPICES', 'COLD', 'SOUR', 'BURNT', 'ACID', 'WARM', 'MUSKY',
       'SWEATY', 'AMMONIA/URINOUS', 'DECAYED', 'WOOD', 'GRASS', 'FLOWER',
       ' CHEMICAL']

y2_ldb = df2.loc[:,cols2] 

In [45]:
print(y2_ldb.shape[1])

21


In [46]:
y1_ldb.reset_index(inplace = True, drop = True)
y2_ldb.reset_index(inplace = True, drop = True)

Features look up

In [47]:
#Features look up

X_ldb = df2[['#oID']].merge(dragon, left_on = '#oID', right_on = 'CID').iloc[:,1:]


Mask for Nan

In [48]:
#Creating a boolean mask for NaN values
rows_mask = X_ldb.isnull().any(axis = 1)

Filter

In [49]:
#Applying the mask both in X and y dataframes
X_ldb, y1_ldb, y2_ldb = X_ldb.loc[~rows_mask],y1_ldb.loc[~rows_mask], y2_ldb.loc[~rows_mask]

Saving to csv

In [50]:
X_ldb.to_csv('task1/X_ldb.csv', index = False, header = True)
y1_ldb.to_csv('task1/y_ldb.csv', index = False, header = True)
y2_ldb.to_csv('task2/y_ldb.csv', index = False, header = True)

**Test set**

In [9]:
test_df.columns

Index(['Compound Identifier', 'Odor', 'Replicate', 'Intensity', 'Dilution',
       'subject #', 'INTENSITY/STRENGTH', 'VALENCE/PLEASANTNESS', 'BAKERY',
       'SWEET', 'FRUIT', 'FISH', 'GARLIC', 'SPICES', 'COLD', 'SOUR', 'BURNT',
       'ACID', 'WARM', 'MUSKY', 'SWEATY', 'AMMONIA/URINOUS', 'DECAYED', 'WOOD',
       'GRASS', 'FLOWER', 'CHEMICAL'],
      dtype='object')

Features look up

In [22]:
#Features look up
X_test = test_df[['Compound Identifier']].merge(dragon, left_on = 'Compound Identifier', right_on = 'CID').iloc[:,1:]

Targets dataframe

In [23]:
y1_test = test_df[['INTENSITY/STRENGTH','subject #']]

y2_test = test_df[['subject #','VALENCE/PLEASANTNESS', 'BAKERY', 'SWEET', 'FRUIT', 'FISH',
       'GARLIC', 'SPICES', 'COLD', 'SOUR', 'BURNT', 'ACID', 'WARM', 'MUSKY',
       'SWEATY', 'AMMONIA/URINOUS', 'DECAYED', 'WOOD', 'GRASS', 'FLOWER',
       'CHEMICAL']]

y1_test.reset_index(inplace = True, drop = True)
y2_test.reset_index(inplace = True, drop = True)

In [24]:
print(X_test.shape, y1_test.shape, y2_test.shape)

(3381, 4870) (3381, 2) (3381, 21)


In [25]:
#Creating a boolean mask for NaN values
rows_mask = X_test.isnull().any(axis = 1)

In [26]:
#Applying the mask both in X and y dataframes
X_test = X_test[~rows_mask]
y1_test = y1_test[~rows_mask]
y2_test = y2_test[~rows_mask]

In [29]:
#Creating a boolean mask for NaN values in the target set
y2_row_mask = y2_test.isnull().any(axis = 1)

In [30]:
#Applying the mask both in X and y dataframes
X_test = X_test[~y2_row_mask]
y1_test = y1_test[~y2_row_mask]
y2_test = y2_test[~y2_row_mask]

In [31]:
print(X_test.shape, y1_test.shape, y2_test.shape)

(2992, 4870) (2992, 2) (2992, 21)


Saving files to csv

In [34]:
X_test.to_csv('task1/X_test.csv', index = False, header = True)
y1_test.to_csv('task1/y_test.csv', index = False, header = True)
y2_test.to_csv('task2/y_test.csv', index = False, header = True)